In [1]:
!pip -q install transformers accelerate bitsandbytes sentencepiece duckduckgo-search trafilatura beautifulsoup4 gradio==4.44.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.11.0 requires 

In [2]:
import os
import re
import time
import json
import math
import textwrap
import requests
import trafilatura
from bs4 import BeautifulSoup
from typing import List, Dict, Any


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


from duckduckgo_search import DDGS
import gradio as gr

In [3]:
PREFERRED_MODELS = [
"Qwen/Qwen2.5-7B-Instruct", # multilingual, strong
"microsoft/Phi-3-mini-4k-instruct", # lightweight, fast
"mistralai/Mistral-7B-Instruct-v0.2", # robust instruct
]

In [4]:
# ================================
# 0) KURULUM • GPU KONTROL • SMOKE TEST
# ================================
# Colab: "Runtime" → "Change runtime type" → GPU (A100) seçtiğinden emin ol.

!pip -q install transformers accelerate bitsandbytes sentencepiece gradio==4.44.0

import torch, os, sys
print("Torch:", torch.__version__)
!nvidia-smi -L || echo "(GPU görünmüyorsa: Runtime→Change runtime type→GPU)"


Torch: 2.8.0+cu126
GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-0c089b69-d624-87c8-74d0-c567cab02680)


In [5]:
# ================================
# 1) MODEL YÜKLEYİCİ • GÜVENLİ GERİ DÖNÜŞ
# ================================
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import traceback, textwrap, re
from dataclasses import dataclass, field
from typing import List, Dict, Any

# Küçük ve erişilebilir modeller: gating/kvota riski düşük
PREFERRED_MODELS = [
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # tiny, hızlı fallback
    "Qwen/Qwen2.5-3B-Instruct",            # 3B, güçlü
    "microsoft/Phi-3-mini-4k-instruct",    # hafif ve hızlı
]

class LLM:
    def __init__(self, model_candidates: List[str] = None, max_new_tokens=512, temperature=0.7):
        self.model_name = None
        self.pipe = None
        self.tok = None
        self.has_chat_template = False
        self.max_new_tokens = max_new_tokens
        self.temperature = temperature

        candidates = model_candidates or PREFERRED_MODELS
        last_err = None
        for name in candidates:
            # 4-bit dene
            try:
                print(f"[LLM] Loading 4-bit: {name} …")
                tok = AutoTokenizer.from_pretrained(name, use_fast=True, trust_remote_code=True)
                # pad token eksikse eos'u pad olarak ata (yaygın uyarıyı önler)
                if tok.pad_token is None and tok.eos_token is not None:
                    tok.pad_token = tok.eos_token

                mdl = AutoModelForCausalLM.from_pretrained(
                    name,
                    device_map="auto",
                    trust_remote_code=True,
                    load_in_4bit=True,
                )
                # model config'te de pad id ayarla
                if getattr(mdl.config, "pad_token_id", None) is None and tok.pad_token_id is not None:
                    mdl.config.pad_token_id = tok.pad_token_id

                self.pipe = pipeline("text-generation", model=mdl, tokenizer=tok, device_map="auto")
                self.tok = tok
                self.model_name = name
                self.has_chat_template = bool(getattr(tok, "chat_template", None))
                print(f"[LLM] Ready (4-bit): {name} | chat_template={self.has_chat_template}")
                break
            except Exception as e:
                print(f"[LLM] 4-bit failed for {name} -> {e}")
                last_err = e
                # fp16'a düş
                try:
                    print(f"[LLM] Loading fp16: {name} …")
                    tok = AutoTokenizer.from_pretrained(name, use_fast=True, trust_remote_code=True)
                    if tok.pad_token is None and tok.eos_token is not None:
                        tok.pad_token = tok.eos_token

                    mdl = AutoModelForCausalLM.from_pretrained(
                        name,
                        device_map="auto",
                        trust_remote_code=True,
                        torch_dtype=torch.float16,
                    )
                    if getattr(mdl.config, "pad_token_id", None) is None and tok.pad_token_id is not None:
                        mdl.config.pad_token_id = tok.pad_token_id

                    self.pipe = pipeline("text-generation", model=mdl, tokenizer=tok, device_map="auto")
                    self.tok = tok
                    self.model_name = name
                    self.has_chat_template = bool(getattr(tok, "chat_template", None))
                    print(f"[LLM] Ready (fp16): {name} | chat_template={self.has_chat_template}")
                    break
                except Exception as e2:
                    print(f"[LLM] fp16 failed for {name} -> {e2}")
                    last_err = e2
                    continue
        if self.pipe is None:
            raise RuntimeError(f"Could not load any model. Last error: {last_err}")

    def chat(self, system: str, user: str, max_new_tokens: int = None, temperature: float = None) -> str:
        max_new_tokens = max_new_tokens or self.max_new_tokens
        temperature = self.temperature if temperature is None else temperature

        # Chat template varsa onu kullan; yoksa eski şablon
        if self.has_chat_template:
            messages = [
                {"role": "system", "content": system},
                {"role": "user", "content": user},
            ]
            prompt = self.tok.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
            split_fallback = False  # prompt'u eklemeyeceğiz, o yüzden bölme gerekmeyecek
        else:
            prompt = f"System:\n{system}\n\nUser:\n{user}\n\nAssistant:"
            split_fallback = True

        out = self.pipe(
            prompt,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=0.95,
            repetition_penalty=1.08,
            eos_token_id=self.tok.eos_token_id if getattr(self.tok, "eos_token_id", None) is not None else None,
            pad_token_id=self.tok.pad_token_id if getattr(self.tok, "pad_token_id", None) is not None else None,
            return_full_text=False,  # sadece üretimi döndür
        )[0]["generated_text"]

        if split_fallback:
            # Bazı sürümlerde return_full_text yine de True davranabilir; yine de güvenli bölme
            return out.split("Assistant:")[-1].strip() if "Assistant:" in out else out.strip()
        return out.strip()

# Smoke test: minik üretim
_llm_smoke = LLM([PREFERRED_MODELS[0]], max_new_tokens=64)
print(_llm_smoke.chat("You are concise.", "Give me one bullet about multi-agent LLMs."))


[LLM] Loading 4-bit: TinyLlama/TinyLlama-1.1B-Chat-v1.0 …


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


[LLM] Ready (4-bit): TinyLlama/TinyLlama-1.1B-Chat-v1.0 | chat_template=True
Multi-agent language models (MLMs) use a large number of agents, each with its own copy of the LM corpus, to generate responses to a user's input text. These agents collaborate in real-time to produce a coherent and natural-sounding output. Multi-agent


In [6]:
# ================================
# 2) PARSING YARDIMCILARI
# ================================
import re
from typing import List

# Genel başlıklar (sınır tespiti için). titles parametresiyle birleştirilecek.
_DEFAULT_HEADERS = [
    "Executive Summary",
    "Summary",
    "Recommendations",
    "Strategic Recommendations",
    "Table",
]

def _normalize(text: str) -> str:
    """Satır sonlarını normalize et, gereksiz boşlukları sadeleştir."""
    if not isinstance(text, str):
        text = str(text)
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    return text

def extract_section(text: str, titles: List[str]) -> str:
    """
    'Title' veya 'Title:' başlığından sonraki bloğu alır.
    Başlık satırı: opsiyonel Markdown işaretleri (#, *, >) + Title[:]
    Bitiş: sonraki bilinen başlık satırı ya da metin sonu.
    Bulunamazsa ilk dolu bloğa düşer.
    """
    text = _normalize(text)
    # Aranan ve sınır başlıkları
    titles = titles or []
    wanted = "|".join(re.escape(t) for t in titles)
    boundaries = list({*_DEFAULT_HEADERS, *titles})
    boundary_alt = "|".join(re.escape(t) for t in boundaries)

    # Başlangıç: tam bir başlık satırı
    start_re = re.compile(
        rf"(?im)^[ \t]*(?:#{1,6}\s*|>+\s*|\*+\s*)?(?:{wanted})\s*:?\s*$"
    )
    m = start_re.search(text)
    if m:
        start = m.end()
        # Sıradaki başlık veya metin sonu
        next_re = re.compile(
            rf"(?im)^[ \t]*(?:#{1,6}\s*|>+\s*|\*+\s*)?(?:{boundary_alt})\s*:?\s*$"
        )
        n = next_re.search(text, start)
        section = text[start : n.start()] if n else text[start:]
        return section.strip()

    # Alternatif: kalın başlık (ör. **Executive Summary:**)
    bold_re = re.compile(
        rf"(?is)(?:\*\*|__)\s*(?:{wanted})\s*:?\s*(?:\*\*|__)\s*[\n]+(.*?)(?=\n(?:\*\*|__)|$)"
    )
    b = bold_re.search(text)
    if b:
        return b.group(1).strip()

    # Fallback: ilk dolu blok
    blocks = [b.strip() for b in re.split(r"\n{2,}", text) if b.strip()]
    return blocks[0] if blocks else text.strip()

def extract_list(text: str, titles: List[str]) -> List[str]:
    """
    Önce ilgili bölümden, yoksa tüm metinden listeleri çeker.
    Desteklenen maddeler: -, *, •, ·, –, —, 1. / 1) / a. / a) / i. (Roma)
    Checkbox biçimleri [ ] / [x] da temizlenir.
    """
    section = extract_section(text, titles)
    lines: List[str] = []
    bullet_re = re.compile(
        r"""^(
             (\d+|[ivxlcdm]+|[A-Za-z])[\.\)]   # 1. / 1) / a. / a) / i.
            |[-*•·–—]                          # -,*,•,·,–,—
        )\s+""",
        re.IGNORECASE | re.VERBOSE,
    )
    for ln in section.splitlines():
        s = ln.strip()
        # Markdown checkbox'ları temizle
        s = re.sub(r"^\[(?: |x|X)\]\s*", "", s)
        if bullet_re.match(s):
            s = bullet_re.sub("", s).strip()
            if s:
                lines.append(s)

    # Bölümde bulunamazsa tüm metne bak
    if not lines:
        for ln in _normalize(text).splitlines():
            s = ln.strip()
            s = re.sub(r"^\[(?: |x|X)\]\s*", "", s)
            if bullet_re.match(s):
                s = bullet_re.sub("", s).strip()
                if s:
                    lines.append(s)

    # Yinelenenleri sırayı koruyarak kaldır, çok uzun satırları filtrele
    seen, uniq = set(), []
    for s in lines:
        if len(s) > 0 and len(s) <= 500 and s not in seen:
            seen.add(s)
            uniq.append(s)
    return uniq[:10]

def extract_table_md(text: str) -> str:
    """
    Markdown tabloyu yakalar.
    Önce ```code fence``` içindeki tabloları, sonra düz tabloyu arar.
    """
    text = _normalize(text)

    # 1) Kod bloğu içinde tablo
    m = re.search(
        r"```(?:markdown|md|table)?\s*([\s\S]*?\|[-:| ]+\|[\s\S]*?)```",
        text,
        flags=re.IGNORECASE,
    )
    if m:
        candidate = m.group(1)
        m2 = re.search(r"\|.+\|\n\|[-:| ]+\|[\s\S]*?(?:\n\n|$)", candidate)
        if m2:
            return m2.group(0).strip()

    # 2) Düz tablo (header + separator zorunlu)
    m = re.search(r"\|.+\|\n\|[-:| ]+\|[\s\S]*?(?:\n\n|$)", text)
    return m.group(0).strip() if m else ""

def extract_last_paragraph(text: str) -> str:
    """
    Son paragraf benzeri bloğu döndürür; tablo/başlık/kod bloklarını yok saymaya çalışır.
    """
    text = _normalize(text)
    # Kod bloklarını ve tablolara ait çizgileri en sonda görmezden gel
    blocks = [b.strip() for b in re.split(r"\n{2,}", text) if b.strip()]
    for chunk in reversed(blocks):
        if chunk.startswith("```") or chunk.startswith("|"):
            continue
        # Çok kısa tek satır başlıkları atla
        if re.match(r"^(#{1,6}\s+|\*\*.+\*\*\s*$)", chunk):
            continue
        return chunk
    return blocks[-1] if blocks else text.strip()


In [7]:
# ================================
# 3) HAFIZA ve AJANLAR
# ================================
import re
from dataclasses import dataclass, field
from typing import List, Dict, Any

@dataclass
class SharedMemory:
    topic: str = ""
    acceptance: str = ""
    subtopics: List[str] = field(default_factory=list)
    findings: List[Dict[str, str]] = field(default_factory=list)  # {source_title, url, note}
    summary: str = ""
    table: str = ""
    recommendations: List[str] = field(default_factory=list)
    log: List[Dict[str, str]] = field(default_factory=list)

    def log_add(self, role: str, content: str):
        self.log.append({"role": role, "content": content})

def _wordcap(text: str, max_words: int = 300) -> str:
    """Metni en fazla max_words olacak şekilde kısaltır."""
    words = text.split()
    return " ".join(words[:max_words]) if len(words) > max_words else text

MANAGER_SYSTEM = (
    "You are the Manager. Define a short plan and strict acceptance criteria "
    "(summary ≤300 words in bullets, 3 actionable recommendations, ≤5-column table)."
)

# Writer'a kesin başlıklar zorunlu (LinkedIn yok)
WRITER_SYSTEM = """
You are the Writer agent. Synthesize the findings and output using the EXACT section headers.

Executive Summary:
- Bullet points only (≤300 words total)

Recommendations:
1. Actionable item
2. Actionable item
3. Actionable item

Table:
| Metric | Value |
|---|---|
| Example | N/A |

Constraints: Executive Summary ≤300 words; Table ≤5 columns; cite short source titles in parentheses when helpful.
""".strip()

CRITIC_SYSTEM = (
    "You are the Critic. Check sourcing coverage, clarity, contradictions, and length limits. "
    "Return a brief bullet list of issues + pass/fail note."
)

class Manager:
    def __init__(self, llm: LLM, memory: SharedMemory):
        self.llm = llm
        self.mem = memory

    def kickoff(self, topic: str):
        self.mem.topic = topic
        ans = self.llm.chat(MANAGER_SYSTEM, f"Topic: {topic}\nGive plan + acceptance criteria in 6 bullets.")
        self.mem.acceptance = ans
        self.mem.log_add("Manager", ans)

class ResearcherLLMOnly:
    """
    Web'e ihtiyaç duymadan (internet gereksiz) 'bulgu listesi' üretir.
    LLM'den 3–5 alt başlık ve her biri için 1 cümlelik not ister.
    """
    SYSTEM = (
        "You are the Researcher. Propose 3–5 subtopics and give 1-sentence factual notes "
        "for each subtopic. Pretend there are short source titles in parentheses."
    )
    def __init__(self, llm: LLM, memory: SharedMemory):
        self.llm = llm
        self.mem = memory

    def _parse_block(self, text: str, start_label: str, end_label: str = None) -> str:
        """Başlığa göre (case-insensitive) blok alır."""
        flags = re.IGNORECASE | re.MULTILINE | re.DOTALL
        if end_label:
            m = re.search(rf'^{start_label}\s*:?\s*\n(.*?)(?=^{end_label}\s*:?\s*$|$)', text, flags)
        else:
            m = re.search(rf'^{start_label}\s*:?\s*\n(.*)$', text, flags)
        return (m.group(1) if m else "").strip()

    def _clean_bullets(self, lines: List[str]) -> List[str]:
        out = []
        bullet_re = re.compile(r'^(\d+[\.\)]|[A-Za-z][\.\)]|[-*•·–—])\s+')
        for ln in lines:
            s = ln.strip()
            s = re.sub(r'^\[(?: |x|X)\]\s*', '', s)  # checkboxları temizle
            s = bullet_re.sub('', s).strip()
            if s and s.lower() not in {"subtopics", "findings"}:
                out.append(s)
        # yinelenenleri kaldır
        seen, uniq = set(), []
        for s in out:
            if s not in seen:
                seen.add(s)
                uniq.append(s[:500])
        return uniq

    def produce_findings(self):
        topic = self.mem.topic
        user = f"Topic: {topic}\nGive 'Subtopics' and 'Findings' sections in plain text."
        out = self.llm.chat(self.SYSTEM, user, max_new_tokens=400)
        self.mem.log_add("Researcher", out)

        # Alt başlıklar
        subs_block = self._parse_block(out, "Subtopics", "Findings")
        sub_lines = subs_block.splitlines() if subs_block else []
        subs = self._clean_bullets(sub_lines)[:5]
        self.mem.subtopics = subs

        # Findings
        findings_block = self._parse_block(out, "Findings") or out
        note_lines = [ln for ln in findings_block.splitlines() if ln.strip()]
        notes_clean = self._clean_bullets(note_lines)
        notes: List[Dict[str, str]] = []
        for s in notes_clean:
            # Sözde kaynak başlığını çıkar (parantez içi varsa), yoksa 'General Source'
            stitle = (s.split("(")[-1].split(")")[0] if "(" in s and ")" in s else "General Source")
            notes.append({
                "source_title": stitle[:80],
                "url": "",
                "note": _wordcap(s, 80)  # notu makul kısa tut
            })
        if not notes:
            notes = [{"source_title": "General Source", "url": "", "note": f"High-level observation about {topic} (General Source)."}]
        self.mem.findings = notes[:8]

class Writer:
    def __init__(self, llm: LLM, memory: SharedMemory):
        self.llm = llm
        self.mem = memory

    def produce(self):
        topic = self.mem.topic
        findings = self.mem.findings
        # findings yoksa, sıkıntı çıkmasın
        if not findings:
            findings = [{"source_title": "General Source", "note": f"Context on {topic}", "url": ""}]
        findings_txt = "\n".join([f"- {f['source_title']}: {f['note']}" for f in findings])

        user = (
            f"Topic: {topic}\nFindings:\n{findings_txt}\n"
            "Create the three artifacts using EXACT headers: Executive Summary:, Recommendations:, Table:."
        )
        out = self.llm.chat(WRITER_SYSTEM, user, max_new_tokens=900)
        self.mem.log_add("Writer", out)

        # --- Executive Summary ---
        summary = extract_section(out, ["Executive Summary", "Summary"])
        if not summary or not summary.strip():
            bullets = [ln.strip() for ln in out.splitlines() if ln.strip().startswith(("-", "*", "•"))]
            summary = "\n".join(bullets[:8]) if bullets else out[:1200]
        self.mem.summary = _wordcap(summary, 300)

        # --- Recommendations ---
        recs = extract_list(out, ["Recommendations", "Strategic Recommendations"]) or []
        if not recs:
            nums = [ln.strip() for ln in out.splitlines() if re.match(r"^\d+[\.\)]\s+", ln.strip())]
            recs = [re.sub(r"^\d+[\.\)]\s+", "", x).strip() for x in nums][:3]
        # boş maddeleri at
        self.mem.recommendations = [r for r in recs if r][:3] or [
            "Define KPIs and baseline metrics.",
            "Prioritize the highest-impact use cases.",
            "Set a 4–6 week pilot plan with owners."
        ]

        # --- Table ---
        table_md = extract_table_md(out)
        if not table_md:
            # Bulgu tabanlı basit bir tablo üret
            rows = "\n".join([f"| {f['source_title']} | {f['note'][:80]} |" for f in findings[:5]])
            table_md = "| Source | Takeaway |\n|---|---|\n" + rows if rows else "| Metric | Value |\n|---|---|\n| Example | N/A |"
        self.mem.table = table_md

class Critic:
    def __init__(self, llm: LLM, memory: SharedMemory):
        self.llm = llm
        self.mem = memory

    def review(self) -> Dict[str, Any]:
        pkg = (
            f"Summary:\n{self.mem.summary}\n\n"
            f"Table:\n{self.mem.table}\n\n"
            f"Recommendations:\n- " + "\n- ".join(self.mem.recommendations)
        )
        out = self.llm.chat(CRITIC_SYSTEM, pkg, max_new_tokens=256)
        self.mem.log_add("Critic", out)
        # Basit pass/fail çıkarımı
        lower = out.lower()
        passed = ("no issues" in lower) or ("pass" in lower and "fail" not in lower)
        return {"pass": passed, "notes": out}


In [8]:
# ================================
# 4) ORKESTRASYON
# ================================
from copy import deepcopy
from datetime import datetime, timezone

class Orchestrator:
    """
    Sağlamlaştırılmış yürütücü:
    - Her adım try/except ile izole: hata loglanır, akış durmaz.
    - rounds değeri güvenli aralığa sıkıştırılır (1..3).
    - Heuristik kalite kontrol: özet uzunluğu, öneri sayısı, tablo varlığı.
    - Critic turu başarısızsa Writer'a hedefli "fix_hint" ile geri döndürür.
    - Derin kopya ile state döndürür.
    """
    def __init__(self, llm: LLM):
        self.llm = llm
        self.mem = SharedMemory()
        self.manager = Manager(llm, self.mem)
        self.researcher_llm = ResearcherLLMOnly(llm, self.mem)
        self.writer = Writer(llm, self.mem)
        self.critic = Critic(llm, self.mem)

    def reset(self):
        """Belleği sıfırla (aynı LLM ile yeni ajan örnekleri)."""
        self.mem = SharedMemory()
        self.manager = Manager(self.llm, self.mem)
        self.researcher_llm = ResearcherLLMOnly(self.llm, self.mem)
        self.writer = Writer(self.llm, self.mem)
        self.critic = Critic(self.llm, self.mem)

    def _log(self, role: str, msg: str):
        ts = datetime.now(timezone.utc).isoformat(timespec="seconds")
        self.mem.log_add(role, f"[{ts} UTC] {msg}")

    def _heuristic_issues(self) -> list:
        """Basit kalite kontrolleri: özet kısalığı, öneri sayısı, tablo varlığı."""
        issues = []
        summary_words = len(self.mem.summary.split())
        if summary_words == 0:
            issues.append("Executive Summary missing or empty.")
        elif summary_words > 300:
            issues.append(f"Executive Summary too long ({summary_words} words > 300).")

        recs = [r for r in (self.mem.recommendations or []) if r and r.strip()]
        if len(recs) < 3:
            issues.append(f"Only {len(recs)} recommendations; need 3.")

        if "|" not in (self.mem.table or "") or "---" not in (self.mem.table or ""):
            issues.append("Table section missing or malformed (expect Markdown table).")
        return issues

    def run(self, topic: str, rounds: int = 1, use_llm_research: bool = True) -> Dict[str, Any]:
        # Parametreleri normalize et
        topic = (topic or "").strip() or "General topic"
        rounds = max(1, min(int(rounds or 1), 3))

        self._log("Manager", f"Run start · topic='{topic}' · rounds={rounds} · use_llm_research={use_llm_research}")

        # 1) Manager kickoff
        try:
            self.manager.kickoff(topic)
        except Exception as e:
            self._log("Manager", f"ERROR in kickoff: {e}")

        # 2) Researcher (isteğe bağlı)
        try:
            if use_llm_research:
                self.researcher_llm.produce_findings()
            else:
                self.mem.findings = []
                self._log("Researcher", "Skipped LLM research (use_llm_research=False).")
        except Exception as e:
            self._log("Researcher", f"ERROR in produce_findings: {e}")
            # Ara bulgu üret: akış durmasın
            self.mem.findings = [{"source_title": "General Source", "url": "", "note": f"Context on {topic}"}]

        # 3) Writer üretim
        try:
            self.writer.produce()
        except Exception as e:
            self._log("Writer", f"ERROR in produce: {e}")
            # Güvenli varsayılanlar
            self.mem.summary = self.mem.summary or "- Summary unavailable due to an error."
            self.mem.recommendations = self.mem.recommendations or [
                "Define KPIs and baseline metrics.",
                "Prioritize the highest-impact use cases.",
                "Set a 4–6 week pilot plan with owners.",
            ]
            self.mem.table = self.mem.table or "| Metric | Value |\n|---|---|\n| N/A | N/A |"

        # 4) Critic turları (rounds-1 defa) + heuristik
        for i in range(1, rounds):
            try:
                rev = self.critic.review()
            except Exception as e:
                self._log("Critic", f"ERROR in review: {e}")
                break

            heur = self._heuristic_issues()
            passed = bool(rev.get("pass", False)) and (len(heur) == 0)

            if passed:
                self._log("Manager", f"Round {i}: Passed critic and heuristics.")
                break

            # Hedefli düzeltme ipuçları
            hints = []
            if not rev.get("pass", False):
                hints.append("Critic raised issues: " + rev.get("notes", "")[:400])
            if heur:
                hints.append("Heuristic issues: " + "; ".join(heur))

            fix_hint = (
                "Revise concisely to address the following:\n- " +
                "\n- ".join(hints) +
                "\nKeep EXACT headers (Executive Summary:, Recommendations:, Table:)."
            )
            self._log("Manager", f"Round {i}: requesting revision.\n{fix_hint}")

            # Writer revizyonu (hatalara karşı güvenli)
            try:
                self.writer.produce()
            except Exception as e:
                self._log("Writer", f"ERROR in revision produce: {e}")
                break

        self._log("Manager", "Run finished.")
        return deepcopy(self.mem.__dict__)


In [11]:
# ================================
# 5) GRADIO UI
# ================================
import gradio as gr

llm_instance = None
orch_instance = None

def ensure_llm(model_pref: str):
    global llm_instance, orch_instance
    if (llm_instance is None) or (model_pref and llm_instance.model_name != model_pref):
        models = [model_pref] + [m for m in PREFERRED_MODELS if m != model_pref] if model_pref else PREFERRED_MODELS
        llm_instance = LLM(models)
        orch_instance = Orchestrator(llm_instance)
    return llm_instance, orch_instance

def run_pipeline(topic: str, rounds: int, model_pref: str, use_research: bool):
    """
    Full pipeline -> 4 Markdown çıktısı (log, summary, table, recs).
    Hata olursa stack trace'i tüm tablara döndürür.
    """
    try:
        _, orch = ensure_llm(model_pref)
        state = orch.run(topic=(topic or "").strip(), rounds=rounds, use_llm_research=use_research)

        # Agent log markdown
        md_blocks = []
        for entry in state.get("log", []):
            role = entry.get("role", "?")
            content = str(entry.get("content", ""))
            block = f"### {role}\n\n> " + content.replace("\n", "\n> ") + "\n"
            md_blocks.append(block)
        log_md = "\n".join(md_blocks) if md_blocks else "_no log_"

        # Final outputs (safe defaults)
        summary_md = (state.get("summary") or "").strip() or "_no summary produced_"
        table_md   = (state.get("table") or "").strip()   or "| Metric | Value |\n|---|---|\n| N/A | N/A |"
        recs_list  = state.get("recommendations") or []
        recs_md    = "\n".join(f"- {str(r)}" for r in recs_list) or "- _no recommendations parsed_"

        # 4 değer döndür (UI outputs ile birebir)
        return log_md, summary_md, table_md, recs_md

    except Exception:
        tb = traceback.format_exc()
        err_md = "**Backend Error**\n\n```\n" + tb + "\n```"
        # HATA DURUMUNDA DA 4 DEĞER DÖNDÜR (önceki sürümde 5'ti → Gradio mismatch)
        return err_md, err_md, err_md, err_md

with gr.Blocks(title="Multi-Agent LLM • Colab (A100)") as demo:
    gr.Markdown("""
    # 🤖 Multi-Agent LLM Mini Scenario
    **Agents:** Manager · Researcher (LLM-only) · Writer · Critic
    **Outputs:** Executive summary, table, 3 recs
    """)

    with gr.Row():
        with gr.Column(scale=1):
            topic_in = gr.Textbox(label="Topic", value="Road anomaly detection trends for traffic safety (2023–2025)")
            rounds_in = gr.Slider(1, 3, value=2, step=1, label="Max Rounds (Critic)")
            model_in = gr.Dropdown(choices=PREFERRED_MODELS, value=PREFERRED_MODELS[0], label="Preferred Model")
            use_research_ck = gr.Checkbox(value=True, label="Use LLM-only Research (no web)")
            run_btn = gr.Button("🚀 Run Scenario")
        with gr.Column(scale=2):
            log_out = gr.Markdown(label="Agent Log")

    with gr.Row():
        with gr.Tab("Executive Summary"):
            summary_out = gr.Markdown()
        with gr.Tab("Table"):
            table_out = gr.Markdown()
        with gr.Tab("Recommendations"):
            recs_out = gr.Markdown()

    run_btn.click(
        fn=run_pipeline,
        inputs=[topic_in, rounds_in, model_in, use_research_ck],
        outputs=[log_out, summary_out, table_out, recs_out],
    )

print("Ready. Now run: demo.launch()")


Ready. Now run: demo.launch()
Ready. Now run: demo.launch()


In [10]:
# ================================
# 6) UI'YI BAŞLAT
# ================================
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


/usr/local/lib/python3.12/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://cc61beed90d8934bb2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
